In [30]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import datetime as dt
import pandas_ta as ta


from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)
import scripts.functions as fn

from scripts.indicators import resample,polyfit_trend

from binance.client import Client

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [31]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [51]:
environment = 'klines'
#environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(15))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2024-6-01'
	end = '2024-11-24'
	symbol = 'BTCUSDT'
	interval = '1d'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)


#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
df = df[6000:6200]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [52]:
df.ta.mom(length=10, append=True)
df = polyfit_trend(df,column='MOM_10', window=5)
df['pf_change'].describe()

count      185.000000
mean       -68.814245
std        962.753019
min     -12353.203661
25%        -47.652422
50%        -10.655770
75%         39.093013
max       2078.859527
Name: pf_change, dtype: float64

In [53]:
df_to_show = df #[0:20]

indicators = [
     #{'col': 'MA','name': 'MA','color': 'white','row': 1, 'mode':'lines',},
     #{'col': 'long_fbe_0','name': 'long_fbe_0','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'long_fbe_1','name': 'long_fbe_1','color': 'white','row': 1, 'mode':'markers',},
     #{'col': 'fb_2','name': 'fb_2','color': 'red','row': 2, 'mode':'markers',},
     #{'col': 'stop_loss','name': 'stop_loss','color': 'gray','row': 1, 'mode':'markers',},
     #{'col': 'take_profit','name': 'take_profit','color': 'gray','row': 1, 'mode':'markers',},
        ]

    
indicators_out = [
     {'col': 'MOM_10','color': 'yellow','row': 1, 'mode':'bars',},
     {'col': 'pf_change','color': 'yellow','row': 2, 'mode':'bars',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df_to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df_to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,show_volume=False) 
fig.show()
